In [4]:
import pandas as pd
import re
from rapidfuzz import fuzz, process

In [5]:
# Módulo TextCleanerAndMatchFinder: 
"""
Descripción del módulo:
Este módulo realiza la limpieza y normalización de texto en las columnas de un DataFrame, específicamente en las descripciones de productos y proveedores. 
Luego, utiliza la librería RapidFuzz para encontrar coincidencias entre las descripciones de productos y proveedores basadas en su similitud.
"""

# Función de limpieza de texto para una columna específica del DataFrame
def clear_column(column):
    """
    Limpia y normaliza el texto en una columna para eliminar términos irrelevantes y uniformar el formato.
    
    Elimina:
    - Números, unidades y abreviaturas como 'mg', 'ml', 'g', etc.
    - Caracteres especiales como puntos, comas, guiones, etc.
    - Espacios extra y convierte todo a minúsculas.
    - Palabras comunes como 'de', 'la', 'el', 'a', etc.
    - Palabras repetidas y términos relacionados con la variabilidad en dosis/presentación.
    
    Parámetros:
        column (str): Texto a limpiar.

    Retorna:
        str: Texto limpio y normalizado.
    """
    columna_limpia = re.sub(
        r'\b(\d+|mg|comp\.?|blisters?|estuche|x|ml|unidades|marca|botella|caja|referencia|cm|g|m|f\.a\.x|'
        r'jbe\.|sol\.|env\.|lata|vial|pvo\.|sobres|gotero|cápsulas|frascos?|blister|bot\.|mcg|sp|p\.b\.)\b', 
        '', 
        column, 
        flags=re.IGNORECASE
    )
    columna_limpia = re.sub(r'[:\.\-,;()]', '', columna_limpia)
    columna_limpia = re.sub(r'\s+', ' ', columna_limpia).strip()
    columna_limpia = columna_limpia.lower()
    columna_limpia = re.sub(r'\b(?:de|la|el|en|a|y|con|para|del|sobre|por|medicamento|tratamiento|ac)\b', '', columna_limpia).strip()
    columna_limpia = re.sub(r'\b(\w+)\s+\1\b', r'\1', columna_limpia)
    columna_limpia = re.sub(r'\b(\d+mg|\d+g|\d+ml)\b', '', columna_limpia)
    return columna_limpia

# Función para cargar los datos desde un archivo CSV
def load_data(df_archivo):
    """
    Carga un archivo CSV en un DataFrame de pandas.
    
    Parámetros:
        df_archivo (str): Ruta o nombre del archivo CSV.
    
    Retorna:
        DataFrame: DataFrame con los datos cargados, o None si hubo un error.
    """
    try:
        df = pd.read_csv(df_archivo)
        return df
    except Exception as e:
        print(f"Error al cargar el archivo {df_archivo}: {e}")
        return None

# Cargar los datos desde archivos CSV
name_df_provedor = "prueba.csv"  
name_df_productos = "L_STMaestroPrd.csv"  

# Cargar los DataFrames
df_proveedor = load_data(name_df_provedor)
df_productos = load_data(name_df_productos)

# Copias de los DataFrames
copy_dfProductos = df_productos.loc[:, df_productos.notna().any()].copy()  # Eliminar columnas vacías
copy_dfProveedores = df_proveedor.copy()

# Limpiar las descripciones en ambas tablas
copy_dfProductos['descrip1_limpia'] = copy_dfProductos['descrip1'].fillna('').apply(clear_column)
copy_dfProveedores['descripcion_limpia'] = copy_dfProveedores['DESCRIPCIÓN'].fillna('').apply(clear_column)

# Crear índice para búsqueda rápida
proveedor_index = copy_dfProveedores['descripcion_limpia'].tolist()

# Función para encontrar la mejor coincidencia entre descripciones de productos y proveedores
def find_best_match(product_desc, proveedor_index, threshold=80):
    """
    Encuentra la mejor coincidencia de una descripción de producto entre las descripciones de proveedores
    utilizando la librería RapidFuzz.
    
    Parámetros:
        product_desc (str): Descripción del producto.
        proveedor_index (list): Lista de descripciones limpias de proveedores.
        threshold (int): Umbral de similitud mínima para considerar una coincidencia.
    
    Retorna:
        tuple: La mejor coincidencia y su porcentaje de similitud si supera el umbral. Si no, devuelve None.
    """
    best_match = process.extractOne(product_desc, proveedor_index, scorer=fuzz.ratio)
    if best_match and best_match[1] >= threshold:
        return best_match[0], best_match[1]
    return None, None

# Comparar las descripciones de los productos con los proveedores
matches = []
no_matches = []

# Iterar por cada producto y encontrar su mejor coincidencia
for _, prod_row in copy_dfProductos.iterrows():
    prod_desc = prod_row['descrip1_limpia']
    match_desc, similarity = find_best_match(prod_desc, proveedor_index)
    if match_desc:
        matches.append({
            'producto': prod_row['descrip1'],
            'proveedor': match_desc,
            'similaridad': similarity
        })
    else:
        no_matches.append(prod_desc)

# Convertir los resultados en DataFrames
df_matches = pd.DataFrame(matches)
df_no_matches = pd.DataFrame(no_matches, columns=['producto_sin_match'])

# Mostrar los resultados
print(f"Coincidencias encontradas: {len(df_matches)}")
print(df_matches)

print(f"\nProductos sin coincidencias: {len(df_no_matches)}")
print(df_no_matches)

C:\Users\OPERADOR\AppData\Local\Temp\ipykernel_11524\3948520167.py:53: DtypeWarning: Columns (7,14,17,18,21,28,39,42,46) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(df_archivo)


Coincidencias encontradas: 105
                                           producto  \
0                    ASPIRINA 4 A 500 mg comp.x 100   
1                      ASPIRINA G 500 mg comp.x 100   
2                       ASPIRINA G 500 mg comp.x 20   
3                            ASPIRINETAS comp.x 100   
4                             ASPIRINETAS comp.x 28   
..                                              ...   
100  SALES PARA REHIDRATACION ORAL sob.x 3 x 27.5 g   
101  SALES PARA REHIDRATACION ORAL sob.x 3 x 27.9 g   
102                SPIRULINA A.M. 300 mg comp.x 100   
103                 SPIRULINA A.M. 500 mg comp.x 60   
104                    SPIRULINA blister x 10 comp.   

                        proveedor  similaridad  
0                        aspirina   100.000000  
1                        aspirina   100.000000  
2                        aspirina   100.000000  
3                        aspirina    84.210526  
4                        aspirina    84.210526  
..            

In [6]:
df_matches

,producto,proveedor,similaridad
0,ASPIRINA 4 A 500 mg comp.x 100,aspirina,100.000000
1,ASPIRINA G 500 mg comp.x 100,aspirina,100.000000
2,ASPIRINA G 500 mg comp.x 20,aspirina,100.000000
3,ASPIRINETAS comp.x 100,aspirina,84.210526
4,ASPIRINETAS comp.x 28,aspirina,84.210526
...,...,...,...
100,SALES PARA REHIDRATACION ORAL sob.x 3 x 27.5 g,sales rehidratacion oral grs,91.228070
101,SALES PARA REHIDRATACION ORAL sob.x 3 x 27.9 g,sales rehidratacion oral grs,91.228070
102,SPIRULINA A.M. 300 mg comp.x 100,aspirina,82.352941
103,SPIRULINA A.M. 500 mg comp.x 60,aspirina,82.352941
